In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('./utils')
from env import load_env_file
from stats import plot_distribution
from tqdm import tqdm

DATA_DIR = load_env_file()

load env file
  root dir:
    /Users/k/Repo/gp-ibd
  current system:
    Darwin
  loaded env file:
    .env.darwin
  loaded data dir:
    /Users/k/Nutstore Files/毕设-EHR/DB
done.


In [4]:
df = pd.read_csv(os.path.join(DATA_DIR, 'outcome', 'admission_stay.csv'))
for _column in ['admittime', 'dischtime', 'edregtime', 'edouttime']:
    # 入院 出院 进入Ed 离开Ed
    df[_column] = pd.to_datetime(df[_column])
    
df['los_ad'] = df['dischtime'] - df['admittime']
df['los_dp'] = df['edouttime'] - df['edregtime']

df.head(2)

,subject_id,hadm_id,admission_type,anchor_year,admittime,dischtime,edregtime,edouttime,deathtime,los_ad,los_dp
0,10098672,21229395.0,EW EMER.,2140,2142-05-16 04:04:00,2142-05-23 14:15:00,2142-05-15 15:49:00,2142-05-16 05:45:00,NaN,7 days 10:11:00,0 days 13:56:00
1,10098672,21259834.0,EW EMER.,2140,2141-04-13 17:30:00,2141-04-17 19:17:00,2141-04-13 12:17:00,2141-04-13 18:47:00,NaN,4 days 01:47:00,0 days 06:30:00


In [13]:
stats = df.groupby('subject_id').agg({
    'los_ad': 'sum',
    'los_dp': 'sum',
})

stats.head()

,los_ad,los_dp
subject_id,,
10001186,16 days 12:06:00,0 days 18:33:00
10007174,4 days 06:26:00,1 days 15:52:00
10018852,12 days 16:21:00,0 days 10:07:00
10024331,97 days 14:06:00,6 days 07:42:00
10025647,59 days 15:40:00,4 days 22:56:00


In [8]:
df_demo = pd.read_csv(os.path.join(DATA_DIR, 'complication', 'Complications_Patients.csv'))


# df_demography = df_demography.loc[p_lists['only_cd'], :]
stats_demo = df_demo.groupby('subject_id').agg({
    'gender': 'first',
    'anchor_age': 'first',
})

stats_demo.head()

,gender,anchor_age
subject_id,,
10001186,F,46
10007174,M,70
10018852,M,19
10024331,M,72
10025647,M,83


In [21]:
df_merge = pd.merge(stats, stats_demo, on='subject_id')

def to_group(age) -> int:
    if age <= 35:
        return 0
    elif age <= 55:
        return 1
    return 2

df_merge['age_group'] = df_merge.anchor_age.apply(to_group)
df_merge.head()

,los_ad,los_dp,gender,anchor_age,age_group
subject_id,,,,,
10001186,16 days 12:06:00,0 days 18:33:00,F,46,1
10007174,4 days 06:26:00,1 days 15:52:00,M,70,2
10018852,12 days 16:21:00,0 days 10:07:00,M,19,0
10024331,97 days 14:06:00,6 days 07:42:00,M,72,2
10025647,59 days 15:40:00,4 days 22:56:00,M,83,2


In [26]:
# By Age
df_merge.groupby('age_group').agg({
    'los_ad': ['mean', 'std'],
    'los_dp': ['mean', 'std'],
})

los_ad                              \
                                 mean                         std   
age_group                                                           
0         103 days 09:35:34.858096828 637 days 20:18:09.922373112   
1         123 days 20:27:25.898734178 984 days 17:56:47.681952784   
2          84 days 16:34:49.844357977 313 days 04:49:15.526305332   

                             los_dp                             
                               mean                        std  
age_group                                                       
0         6 days 05:17:21.035058430 54 days 06:53:55.644382717  
1         6 days 12:21:50.810126582 47 days 14:39:16.641040697  
2         3 days 23:06:30.291828793 16 days 15:22:28.737098979

In [27]:
# By Gender
df_merge.groupby('gender').agg({
    'los_ad': ['mean', 'std'],
    'los_dp': ['mean', 'std'],
})

los_ad                              \
                              mean                         std   
gender                                                           
F      101 days 09:00:07.600596125 551 days 21:33:18.579145832   
M      103 days 01:35:57.767441860 808 days 02:50:10.606630528   

                          los_dp                             
                            mean                        std  
gender                                                       
F      5 days 05:37:33.263785394 39 days 12:37:19.839075658  
M      5 days 12:13:01.451162790 40 days 07:01:55.318754470